# Content Extraction and Analysis Pipeline

This notebook uses the ImprovedContentExtractionAgent to:
1. Extract and analyze news articles for companies
2. Clean and filter results
3. Segment by subsidiary companies
4. Export to Excel format


In [10]:
import pandas as pd
import warnings
from datetime import datetime
from pathlib import Path
import json

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Import the improved content extraction agent
from agents.improved_content_extraction_agent import ImprovedContentExtractionAgent
from agents.alias_agent import agent_executor


: 

## Step 1: Configure Company Information

Define the company aliases, parent company, and search queries


In [11]:
# Target company name - update this for your analysis
target_company = "Adani Enterprises Limited"

print(f"🤖 Generating AI-powered configuration for: {target_company}")
print("🔍 Using alias agent to find company aliases, parent company, and search queries...")

# Use the alias agent to get company information
alias_result = agent_executor.invoke({
    "input": f"What are alias for {target_company} in India, including parent company"
})

print(f"✅ Alias agent response received")
print(f"📊 Response type: {type(alias_result)}")

# Parse the response
if isinstance(alias_result, dict) and 'output' in alias_result:
    try:
        # Try to parse JSON from the output
        company_config = json.loads(alias_result['output'])
        print("✅ Successfully parsed JSON response")
    except json.JSONDecodeError:
        # If JSON parsing fails, try to extract from the text
        print("⚠️  JSON parsing failed, attempting text extraction...")
        # Fallback: create a basic config
        company_config = {
            "primary_alias": target_company,
            "aliases": [target_company],
            "parent_company": "Unknown",
            "adverse_search_querires": [
                f"{target_company} adverse news",
                f"{target_company} fraud",
                f"{target_company} controversy",
                f"{target_company} lawsuit",
                f"{target_company} regulatory issues"
            ]
        }
else:
    print("⚠️  Unexpected response format, using fallback configuration")
    company_config = {
        "primary_alias": target_company,
        "aliases": [target_company],
        "parent_company": "Unknown",
        "adverse_search_querires": [
            f"{target_company} adverse news",
            f"{target_company} fraud",
            f"{target_company} controversy",
            f"{target_company} lawsuit",
            f"{target_company} regulatory issues"
        ]
    }

# Extract key variables
company_aliases = company_config.get("all_aliases", ", ".join(company_config.get("aliases", [target_company])))
parent_company_name = company_config.get("parent_company", "Unknown")
search_queries = company_config.get("adverse_search_querires", [])
primary_company_name = company_config.get("primary_alias", target_company)

print(f"\n🏢 Generated Configuration:")
print(f"   �� Primary Company: {primary_company_name}")
print(f"   🏭 Parent Company: {parent_company_name}")
print(f"   �� Company Aliases: {company_aliases}")
print(f"   📊 Search Queries: {len(search_queries)} queries generated")
print(f"\n🔎 Generated search queries:")
for i, query in enumerate(search_queries[:5]):
    print(f"   {i+1}. {query}")
if len(search_queries) > 5:
    print(f"   ... and {len(search_queries) - 5} more")

# Display full configuration for verification
print(f"\n📋 Full AI-Generated Configuration:")
print(json.dumps(company_config, indent=2))

🤖 Generating AI-powered configuration for: Adani Enterprises Limited
🔍 Using alias agent to find company aliases, parent company, and search queries...


> Entering new AgentExecutor chain...
15:34:31 - httpx - INFO - HTTP Request: POST https://ai-gpt-echo.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
{
  "primary_alias": "Adani Enterprises Limited",
  "aliases": [
    "Adani Enterprises",
    "Adani Enterprises Ltd",
    "Adani Enterprise"
  ],
  "stock_symbols": [
    "ADANIENT"
  ],
  "local_variants": [
    "Adani Enterprises India"
  ],
  "parent_company": "Adani Group",
  "adverse_search_querires": [
    "Adani Enterprises Limited adverse news",
    "Adani Enterprises Limited scandal",
    "Adani Enterprises controversy",
    "Adani Enterprises Ltd fraud",
    "Adani Enterprise lawsuit",
    "Adani Enterprises Limited whistleblower",
    "Adani Enterprises Limited SEC investigation",
    "Adani Group governance issue

## Step 2: Initialize Content Extraction Agent

Create the agent with production-ready settings (no caching)


In [12]:
# Initialize the content extraction agent
extractor = ImprovedContentExtractionAgent(
    num_results=10,  # Number of results per query
    concurrent_limit=24  # Concurrent processing limit
)

print("✅ ImprovedContentExtractionAgent initialized")
print("🚀 Ready for fresh content extraction and analysis")


15:35:05 - agents.improved_content_extraction_agent - INFO - 🚀 Initializing ImprovedContentExtractionAgent with fresh extraction and analysis
15:35:05 - agents.improved_content_extraction_agent - INFO - 📊 Configuration: concurrent_limit=24, num_results=10, use_duckduckgo=True
15:35:05 - agents.improved_content_extraction_agent - INFO - 🔍 DuckDuckGo search enabled
✅ ImprovedContentExtractionAgent initialized
🚀 Ready for fresh content extraction and analysis


## Step 3: Search and Extract Content

Run the content extraction pipeline for all search queries


In [13]:
# Search and extract content for the target company
print(f"🚀 Starting search and content extraction for: {primary_company_name}")

# Use the main search and extract method
results_data = await extractor.search_and_extract_content(
    search_queries=search_queries[:1],
    aliases=company_aliases, 
    parent_company_name=parent_company_name
)

# Extract the results
results = results_data["search_results"]

print(f"✅ Content extraction completed!")
print(f"📊 Total articles processed: {len(results)}")


🚀 Starting search and content extraction for: Adani Enterprises Limited
15:35:27 - agents.improved_content_extraction_agent - INFO - Starting fresh search and content extraction
15:35:27 - agents.improved_content_extraction_agent - INFO - Starting content extraction for queries: ['Adani Enterprises Limited adverse news']
15:35:27 - agents.improved_content_extraction_agent - INFO - Date range: 2024-07-25 to 2025-07-25
15:35:27 - agents.improved_content_extraction_agent - INFO - Num results per query: 10
15:35:27 - agents.improved_content_extraction_agent - INFO - Search engines: Google + DuckDuckGo


Date Windows:   0%|                                                                                           | 0/13 [00:00<?, ?window/s]

15:35:27 - agents.improved_content_extraction_agent - INFO - Searching for period: 2024-07-25 to 2024-08-25



Queries (2024-07-25 to 2024-08-25):   0%|                                                                          | 0/1 [00:00<?, ?it/s]

15:35:27 - agents.improved_content_extraction_agent - INFO - Searching query: Adani Enterprises Limited adverse news
15:35:27 - agents.improved_content_extraction_agent - INFO - Google: Found 10 results for query 'Adani Enterprises Limited adverse news'
15:35:27 - agents.improved_content_extraction_agent - INFO - DuckDuckGo: Fetching 1 pages for query 'Adani Enterprises Limited adverse news' (target: 10 results)
15:35:27 - agents.improved_content_extraction_agent - INFO - DuckDuckGo page 1 (start=0): Found 10 results
15:35:27 - agents.improved_content_extraction_agent - INFO - DuckDuckGo: Total 10 results for query 'Adani Enterprises Limited adverse news'


...

15:37:12 - httpx - INFO - HTTP Request: POST https://ai-gpt-echo.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Processing URLs: 100%|████████████████████████████████████████████████████████████████████████████████▋| 241/242 [01:41<00:02,  2.70s/it]

15:37:14 - httpx - INFO - HTTP Request: POST https://ai-gpt-echo.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Processing URLs: 100%|█████████████████████████████████████████████████████████████████████████████████| 242/242 [01:44<00:00,  2.31it/s]

15:37:14 - agents.improved_content_extraction_agent - INFO - 📊 FINAL STATISTICS:
15:37:14 - agents.improved_content_extraction_agent - INFO -    📋 Total URLs processed: 242
15:37:14 - agents.improved_content_extraction_agent - INFO -    🔄 Content Extraction Results:
15:37:14 - agents.improved_content_extraction_agent - INFO -       ✅ Successful extractions: 242
15:37:14 - agents.improved_content_extraction_agent - INFO -       📭 No content found: 0
15:37:14 - agents.improved_content_extraction_agent - INFO -       ❌ Failed extractions: 0
15:37:14 - agents.improved_content_extraction_agent - INFO -    🤖 AI Analysis Results:
15:37:14 - agents.improved_content_extraction_agent - INFO -       🔍 Successfully analyzed by AI: 239
15:37:14 - agents.improved_content_extraction_agent - INFO -       🏷️  AI-filtered as irrelevant: 47
15:37:14 - agents.improved_content_extraction_agent - INFO -    📈 Overall success rate: 100.0%
15:37:14 - agents.improved_content_extraction_agent - INFO - Completed 

## Step 4: Process Results into DataFrame

Convert the analysis results into a structured DataFrame


In [15]:
def extract_metadata_field(result, field, default=None):
    """Helper to extract from analysis['metadata'] if present, else from analysis"""
    analysis = result.get('analysis', {}) or {}
    metadata = analysis.get('metadata', {}) or {}
    if isinstance(metadata, dict) and field in metadata:
        return metadata.get(field, default)
    if isinstance(analysis, dict):
        return analysis.get(field, default)
    return default

def safe_get_jina_field(result, field, default=''):
    """Safely extract field from jina_content"""
    jina_content = result.get('jina_content', {}) or {}
    return jina_content.get(field, default)

# Convert results to DataFrame
print("📊 Converting results to DataFrame...")

df_data = []
for result in results:
    row = {
        # Basic URL and content info
        'url': result.get('link', ''),
        'title': result.get('title', ''),
        'snippet': result.get('snippet', ''),
        'source': result.get('source', ''),
        'date': result.get('date', ''),
        'source_query': result.get('source_query', ''),
        'search_engine': result.get('search_engine', ''),  # <-- Added search engine
        
        # Content extraction info
        'extraction_status': result.get('extraction_status', 'unknown'),
        'content_title': safe_get_jina_field(result, 'title'),
        'content_length': len(safe_get_jina_field(result, 'content')),
        'content_published_time': safe_get_jina_field(result, 'publishedTime'),
        
        # Analysis results
        'is_adverse': extract_metadata_field(result, 'is_adverse', 'Neutral'),
        'is_adverse_reason': extract_metadata_field(result, 'is_adverse_reason', ''),
        'is_filtered': extract_metadata_field(result, 'is_filter', False),
        'is_filter_reason': extract_metadata_field(result, 'is_filter_reason', ''),
        'published_date': extract_metadata_field(result, 'published_date', ''),
        'author': extract_metadata_field(result, 'author', ''),
        'keywords': extract_metadata_field(result, 'keywords', []),
        
        # Risk metadata
        'risk_category': extract_metadata_field(result, 'risk_category', ''),
        'risk_explanation': extract_metadata_field(result, 'risk_explanation', ''),
        'risk_snippet': extract_metadata_field(result, 'risk_snippet', ''),
        'priority_level': extract_metadata_field(result, 'priority_level', ''),
        'confidence_score': extract_metadata_field(result, 'confidence_score', 0.0),
        'event_timeline': extract_metadata_field(result, 'event_timeline', ''),
        
        # Adverse event flags
        'has_fraud': extract_metadata_field(result, 'has_fraud', False),
        'has_litigation': extract_metadata_field(result, 'has_litigation', False),
        'has_insolvency': extract_metadata_field(result, 'has_insolvency', False),
        'has_regulatory_action': extract_metadata_field(result, 'has_regulatory_action', False),
        
        # Parent/subsidiary relationship
        'is_subsidiary_parent_company': extract_metadata_field(result, 'is_subsadariy_parent_company', False),
        'is_subsidiary_parent_company_reason': extract_metadata_field(result, 'is_subsadariy_parent_company_reason', ''),
    }
    df_data.append(row)

# Create DataFrame
df = pd.DataFrame(df_data)

print(f"✅ DataFrame created with {len(df)} rows and {len(df.columns)} columns")
print(f"📊 Data shape: {df.shape}")

# Display basic statistics
print(f"\n📈 Basic Statistics:")
print(f"   🔄 Extraction Status:")
extraction_stats = df['extraction_status'].value_counts()
for status, count in extraction_stats.items():
    print(f"      {status}: {count} ({count/len(df)*100:.1f}%)")

print(f"   🏷️  Filter Status:")
filter_stats = df['is_filtered'].value_counts()
for status, count in filter_stats.items():
    print(f"      Filtered: {status} = {count} ({count/len(df)*100:.1f}%)")

print(f"   ⚠️  Adverse Status:")
adverse_stats = df['is_adverse'].value_counts()
for status, count in adverse_stats.items():
    print(f"      {status}: {count} ({count/len(df)*100:.1f}%)")


📊 Converting results to DataFrame...
✅ DataFrame created with 242 rows and 30 columns
📊 Data shape: (242, 30)

📈 Basic Statistics:
   🔄 Extraction Status:
      success: 242 (100.0%)
   🏷️  Filter Status:
      Filtered: False = 195 (80.6%)
      Filtered: True = 47 (19.4%)
   ⚠️  Adverse Status:
      Negative: 129 (53.3%)
      Neutral: 73 (30.2%)
      Positive: 40 (16.5%)


## Step 5: Clean and Filter Data

Remove filtered content, failed extractions, and clean the dataset


In [16]:
print(f"🧹 Cleaning and filtering data...")
print(f"📊 Starting with {len(df)} total articles")

# Store original for comparison
original_count = len(df)

# Step 1: Remove failed extractions
df_cleaned = df[df['extraction_status'] == 'success'].copy()
failed_extractions = original_count - len(df_cleaned)
print(f"❌ Removed {failed_extractions} failed extractions")

# Step 2: Remove filtered articles (not relevant to company)
df_cleaned = df_cleaned[df_cleaned['is_filtered'] == False].copy()
filtered_articles = len(df) - failed_extractions - len(df_cleaned)
print(f"🏷️  Removed {filtered_articles} filtered/irrelevant articles")

# Step 3: Parse and clean published dates
print(f"📅 Cleaning published dates...")
def parse_date_robust(date_str):
    """Robust date parsing function"""
    if pd.isna(date_str) or date_str == '':
        return None
    try:
        return pd.to_datetime(date_str, errors='coerce')
    except:
        return None

df_cleaned['published_date'] = df_cleaned['published_date'].apply(parse_date_robust)

# Step 4: Remove articles without valid dates
articles_with_dates = df_cleaned['published_date'].notna().sum()
df_cleaned = df_cleaned[df_cleaned['published_date'].notna()].copy()
articles_without_dates = original_count - len(df_cleaned) - failed_extractions - filtered_articles
print(f"📅 Kept {len(df_cleaned)} articles with valid published dates")
if articles_without_dates > 0:
    print(f"📅 Removed {articles_without_dates} articles without valid dates")

# Step 5: Sort by published date (newest first)
df_cleaned = df_cleaned.sort_values('published_date', ascending=False).reset_index(drop=True)

# Step 6: Clean keywords field
def clean_keywords(keywords):
    """Convert keywords list to string"""
    if isinstance(keywords, list) and keywords:
        return ", ".join([str(k) for k in keywords if k])
    return ""

df_cleaned['keywords'] = df_cleaned['keywords'].apply(clean_keywords)

print(f"\n✅ Data cleaning completed!")
print(f"📊 Final dataset: {len(df_cleaned)} articles")
print(f"📉 Removed: {original_count - len(df_cleaned)} articles total")
if original_count > 0:
    print(f"📈 Retention rate: {len(df_cleaned)/original_count*100:.1f}%")

# Show date range
if len(df_cleaned) > 0:
    date_range = f"{df_cleaned['published_date'].min().date()} to {df_cleaned['published_date'].max().date()}"
    print(f"📅 Date range: {date_range}")
    
    # Show adverse statistics for cleaned data
    print(f"\n⚠️  Final Adverse Content Distribution:")
    final_adverse_stats = df_cleaned['is_adverse'].value_counts()
    for status, count in final_adverse_stats.items():
        print(f"      {status}: {count} articles ({count/len(df_cleaned)*100:.1f}%)")


🧹 Cleaning and filtering data...
📊 Starting with 242 total articles
❌ Removed 0 failed extractions
🏷️  Removed 47 filtered/irrelevant articles
📅 Cleaning published dates...
📅 Kept 190 articles with valid published dates
📅 Removed 5 articles without valid dates

✅ Data cleaning completed!
📊 Final dataset: 190 articles
📉 Removed: 52 articles total
📈 Retention rate: 78.5%
📅 Date range: 2021-06-14 to 2025-07-25

⚠️  Final Adverse Content Distribution:
      Negative: 127 articles (66.8%)
      Positive: 39 articles (20.5%)
      Neutral: 24 articles (12.6%)


## Step 6: Segment by Subsidiary/Parent Company

Create separate datasets for subsidiary-specific vs parent company impacts


In [7]:
print(f"🏢 Segmenting articles by subsidiary vs parent company impact...")

# Segment 1: Articles specifically about the subsidiary/target company
subsidiary_articles = df_cleaned[
    df_cleaned['is_subsidiary_parent_company'] == False
].copy()

# Segment 2: Articles about parent company that impact subsidiary
parent_impact_articles = df_cleaned[
    df_cleaned['is_subsidiary_parent_company'] == True
].copy()

print(f"\n📊 Segmentation Results:")
print(f"   🎯 Subsidiary-specific articles: {len(subsidiary_articles)}")
print(f"   🏭 Parent company impact articles: {len(parent_impact_articles)}")
print(f"   📋 Total: {len(subsidiary_articles) + len(parent_impact_articles)}")

# Show adverse distribution for each segment
if len(subsidiary_articles) > 0:
    print(f"\n🎯 Subsidiary-specific - Adverse Distribution:")
    sub_adverse = subsidiary_articles['is_adverse'].value_counts()
    for status, count in sub_adverse.items():
        print(f"      {status}: {count} articles ({count/len(subsidiary_articles)*100:.1f}%)")

if len(parent_impact_articles) > 0:
    print(f"\n🏭 Parent company impact - Adverse Distribution:")
    parent_adverse = parent_impact_articles['is_adverse'].value_counts()
    for status, count in parent_adverse.items():
        print(f"      {status}: {count} articles ({count/len(parent_impact_articles)*100:.1f}%)")

# Create risk analysis summary
print(f"\n⚠️  Risk Analysis Summary:")
risk_columns = ['has_fraud', 'has_litigation', 'has_insolvency', 'has_regulatory_action']
for risk_type in risk_columns:
    risk_count = df_cleaned[df_cleaned[risk_type] == True][risk_type].sum()
    if risk_count > 0:
        print(f"   ⚡ {risk_type.replace('has_', '').title()}: {risk_count} articles")

# Show priority levels
if 'priority_level' in df_cleaned.columns:
    priority_stats = df_cleaned[df_cleaned['priority_level'] != '']['priority_level'].value_counts()
    if len(priority_stats) > 0:
        print(f"\n🚨 Priority Level Distribution:")
        for priority, count in priority_stats.items():
            print(f"      {priority}: {count} articles")


🏢 Segmenting articles by subsidiary vs parent company impact...

📊 Segmentation Results:
   🎯 Subsidiary-specific articles: 66
   🏭 Parent company impact articles: 127
   📋 Total: 193

🎯 Subsidiary-specific - Adverse Distribution:
      Positive: 34 articles (51.5%)
      Negative: 17 articles (25.8%)
      Neutral: 15 articles (22.7%)

🏭 Parent company impact - Adverse Distribution:
      Negative: 112 articles (88.2%)
      Positive: 9 articles (7.1%)
      Neutral: 6 articles (4.7%)

⚠️  Risk Analysis Summary:
   ⚡ Fraud: 58 articles
   ⚡ Litigation: 62 articles
   ⚡ Regulatory_Action: 92 articles

🚨 Priority Level Distribution:
      High: 79 articles
      Medium: 44 articles
      Low: 5 articles


## Step 7: Export to Excel

Save the results to Excel files with multiple sheets


In [17]:
# Create output directory
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Generate timestamp for filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
company_safe_name = primary_company_name.replace(" ", "_").replace(",", "")

# Define output files
excel_file = output_dir / f"{company_safe_name}_analysis_{timestamp}.xlsx"
csv_file = output_dir / f"{company_safe_name}_analysis_{timestamp}.csv"

print(f"💾 Exporting results to Excel and CSV...")
print(f"📁 Output directory: {output_dir.absolute()}")

# Prepare data for export
export_columns = [
    'url', 'title', 'published_date', 'is_adverse', 'is_adverse_reason',
    'risk_category', 'risk_explanation', 'risk_snippet', 'priority_level',
    'confidence_score', 'has_fraud', 'has_litigation', 'has_insolvency',
    'has_regulatory_action', 'author', 'keywords', 'source_query',
    'is_subsidiary_parent_company', 'is_subsidiary_parent_company_reason',
    'content_length', 'event_timeline'
]

# Ensure all export columns exist
for col in export_columns:
    if col not in df_cleaned.columns:
        df_cleaned[col] = ''

# Export to Excel with multiple sheets
with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    # Sheet 1: All cleaned articles
    df_cleaned[export_columns].to_excel(
        writer, 
        sheet_name='All_Articles', 
        index=False
    )
    
    # Sheet 2: Subsidiary-specific articles
    if len(subsidiary_articles) > 0:
        subsidiary_articles[export_columns].to_excel(
            writer, 
            sheet_name='Subsidiary_Specific', 
            index=False
        )
    
    # Sheet 3: Parent company impact articles
    if len(parent_impact_articles) > 0:
        parent_impact_articles[export_columns].to_excel(
            writer, 
            sheet_name='Parent_Company_Impact', 
            index=False
        )
    
    # Sheet 4: Adverse articles only
    adverse_articles = df_cleaned[df_cleaned['is_adverse'] == 'Negative']
    if len(adverse_articles) > 0:
        adverse_articles[export_columns].to_excel(
            writer, 
            sheet_name='Adverse_Only', 
            index=False
        )
    
    # Sheet 5: Summary statistics
    summary_data = [
        ['Company Name', primary_company_name],
        ['Parent Company', parent_company_name],
        ['Analysis Date', datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
        ['Total Articles Processed', len(df)],
        ['Articles After Cleaning', len(df_cleaned)],
        ['Subsidiary-Specific Articles', len(subsidiary_articles)],
        ['Parent Company Impact Articles', len(parent_impact_articles)],
        ['Adverse Articles', len(adverse_articles)],
        ['', ''],
        ['Adverse Distribution', ''],
    ]
    
    # Add adverse statistics
    if len(df_cleaned) > 0:
        adverse_stats = df_cleaned['is_adverse'].value_counts()
        for status, count in adverse_stats.items():
            summary_data.append([f'{status} Articles', f'{count} ({count/len(df_cleaned)*100:.1f}%)'])
    
    summary_df = pd.DataFrame(summary_data, columns=['Metric', 'Value'])
    summary_df.to_excel(
        writer, 
        sheet_name='Summary', 
        index=False
    )

# Export main dataset to CSV
df_cleaned[export_columns].to_csv(csv_file, index=False)

print(f"\n✅ Export completed successfully!")
print(f"📊 Excel file: {excel_file.name}")
print(f"📋 CSV file: {csv_file.name}")
print(f"\n📈 Export Summary:")
print(f"   📄 All Articles sheet: {len(df_cleaned)} rows")
print(f"   🎯 Subsidiary Specific sheet: {len(subsidiary_articles)} rows")
print(f"   🏭 Parent Company Impact sheet: {len(parent_impact_articles)} rows")
print(f"   ⚠️  Adverse Only sheet: {len(adverse_articles)} rows")
print(f"   📊 Summary sheet: Analysis metadata")

# Show file sizes
if excel_file.exists():
    excel_size = excel_file.stat().st_size / (1024 * 1024)  # MB
    csv_size = csv_file.stat().st_size / (1024 * 1024)  # MB
    print(f"\n💾 File Sizes:")
    print(f"   📊 Excel: {excel_size:.2f} MB")
    print(f"   📋 CSV: {csv_size:.2f} MB")


💾 Exporting results to Excel and CSV...
📁 Output directory: /home/bereket/Downloads/pulse_v2/output

✅ Export completed successfully!
📊 Excel file: Adani_Enterprises_Limited_analysis_20250725_153722.xlsx
📋 CSV file: Adani_Enterprises_Limited_analysis_20250725_153722.csv

📈 Export Summary:
   📄 All Articles sheet: 190 rows
   🎯 Subsidiary Specific sheet: 66 rows
   🏭 Parent Company Impact sheet: 127 rows
   ⚠️  Adverse Only sheet: 127 rows
   📊 Summary sheet: Analysis metadata

💾 File Sizes:
   📊 Excel: 0.23 MB
   📋 CSV: 0.25 MB


## Step 8: Final Analysis Summary

Display key insights and recommendations


In [18]:
print("📋 FINAL ANALYSIS SUMMARY")
print("=" * 50)

print(f"\n🏢 Company: {primary_company_name}")
print(f"🏭 Parent: {parent_company_name}")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n📊 PROCESSING RESULTS:")
print(f"   ✅ Successfully analyzed: {len(df_cleaned)}")

if len(df_cleaned) > 0:
    print(f"\n⚠️  RISK ASSESSMENT:")
    
    # Adverse content
    adverse_count = len(df_cleaned[df_cleaned['is_adverse'] == 'Negative'])
    positive_count = len(df_cleaned[df_cleaned['is_adverse'] == 'Positive'])
    neutral_count = len(df_cleaned[df_cleaned['is_adverse'] == 'Neutral'])
    
    print(f"   🚨 Adverse articles: {adverse_count} ({adverse_count/len(df_cleaned)*100:.1f}%)")
    print(f"   ✅ Positive articles: {positive_count} ({positive_count/len(df_cleaned)*100:.1f}%)")
    print(f"   ⚪ Neutral articles: {neutral_count} ({neutral_count/len(df_cleaned)*100:.1f}%)")
    
    # Risk categories
    risk_types = {
        'has_fraud': '💰 Fraud',
        'has_litigation': '⚖️  Litigation',
        'has_insolvency': '💸 Insolvency',
        'has_regulatory_action': '🏛️  Regulatory'
    }
    
    print(f"\n🚨 SPECIFIC RISK TYPES:")
    for risk_col, risk_name in risk_types.items():
        if risk_col in df_cleaned.columns:
            risk_count = df_cleaned[df_cleaned[risk_col] == True][risk_col].sum()
            if risk_count > 0:
                print(f"   {risk_name}: {risk_count} articles")
    
    # Priority levels
    high_priority = len(df_cleaned[df_cleaned['priority_level'] == 'High'])
    medium_priority = len(df_cleaned[df_cleaned['priority_level'] == 'Medium'])
    low_priority = len(df_cleaned[df_cleaned['priority_level'] == 'Low'])
    
    if high_priority + medium_priority + low_priority > 0:
        print(f"\n🎯 PRIORITY DISTRIBUTION:")
        if high_priority > 0:
            print(f"   🔴 High priority: {high_priority} articles")
        if medium_priority > 0:
            print(f"   🟡 Medium priority: {medium_priority} articles")
        if low_priority > 0:
            print(f"   🟢 Low priority: {low_priority} articles")
    
    # Date range
    date_min = df_cleaned['published_date'].min()
    date_max = df_cleaned['published_date'].max()
    print(f"\n📅 TIME RANGE:")
    print(f"   📅 From: {date_min.strftime('%Y-%m-%d')}")
    print(f"   📅 To: {date_max.strftime('%Y-%m-%d')}")
    print(f"   📊 Span: {(date_max - date_min).days} days")
    
    # Company relationship analysis
    subsidiary_specific = len(df_cleaned[df_cleaned['is_subsidiary_parent_company'] == False])
    parent_impact = len(df_cleaned[df_cleaned['is_subsidiary_parent_company'] == True])
    
    print(f"\n🏢 COMPANY RELATIONSHIP ANALYSIS:")
    print(f"   🎯 Direct subsidiary impact: {subsidiary_specific} articles ({subsidiary_specific/len(df_cleaned)*100:.1f}%)")
    print(f"   🏭 Parent company spillover: {parent_impact} articles ({parent_impact/len(df_cleaned)*100:.1f}%)")

print(f"\n💾 OUTPUT FILES:")
print(f"   📊 Excel: {excel_file.name}")
print(f"   📋 CSV: {csv_file.name}")
print(f"   📁 Location: {output_dir.absolute()}")

print(f"\n🎉 Analysis completed successfully!")
print("=" * 50)


📋 FINAL ANALYSIS SUMMARY

🏢 Company: Adani Enterprises Limited
🏭 Parent: Adani Group
📅 Analysis Date: 2025-07-25 15:37:46

📊 PROCESSING RESULTS:
   ✅ Successfully analyzed: 190

⚠️  RISK ASSESSMENT:
   🚨 Adverse articles: 127 (66.8%)
   ✅ Positive articles: 39 (20.5%)
   ⚪ Neutral articles: 24 (12.6%)

🚨 SPECIFIC RISK TYPES:
   💰 Fraud: 59 articles
   ⚖️  Litigation: 52 articles
   🏛️  Regulatory: 92 articles

🎯 PRIORITY DISTRIBUTION:
   🔴 High priority: 83 articles
   🟡 Medium priority: 38 articles
   🟢 Low priority: 4 articles

📅 TIME RANGE:
   📅 From: 2021-06-14
   📅 To: 2025-07-25
   📊 Span: 1502 days

🏢 COMPANY RELATIONSHIP ANALYSIS:
   🎯 Direct subsidiary impact: 65 articles (34.2%)
   🏭 Parent company spillover: 125 articles (65.8%)

💾 OUTPUT FILES:
   📊 Excel: Adani_Enterprises_Limited_analysis_20250725_153722.xlsx
   📋 CSV: Adani_Enterprises_Limited_analysis_20250725_153722.csv
   📁 Location: /home/bereket/Downloads/pulse_v2/output

🎉 Analysis completed successfully!
